In [1]:
import numpy as np
import cv2
from master_lib.utils import files_and_directories
from os.path import join

In [2]:
def search_train_key(key, train_path):
    subdirs = files_and_directories.list_dirs(train_path)
    for subdir in subdirs:
        subsubdir = files_and_directories.list_dirs(join(train_path, subdir))[0]
        file_name = files_and_directories.list_spec_files(join(train_path, subdir, subsubdir), '.dcm')[0]
        if file_name == key:
            return join(train_path, subdir, subsubdir, file_name)

    return None

In [3]:
dataset_path = r'C:\Users\MikołajStryja\Documents\Studia\master\dataset.json'
master_img_path = r'C:\Users\MikołajStryja\Documents\Studia\do_magisterki_obrazy'

In [4]:
dataset = files_and_directories.load_json(dataset_path)

In [5]:
keys = list(dataset.keys())

In [6]:
images_path = r'C:\Users\MikołajStryja\Documents\Studia\siim-covid19-detection\train'
A = search_train_key(keys[0], images_path)

In [7]:
A

'C:\\Users\\MikołajStryja\\Documents\\Studia\\siim-covid19-detection\\train\\5776db0cec75\\81456c9c5423\\000a312787f2.dcm'

In [8]:
A_metadata = dataset[keys[0]]
A_metadata

{'Atypical Appearance': 0,
 'Indeterminate Appearance': 0,
 'Negative for Pneumonia': 0,
 'StudyInstanceUID': '5776db0cec75',
 'Typical Appearance': 1,
 'boxes': [{'height': 1917.30292,
   'width': 1026.65662,
   'x': 789.28836,
   'y': 582.43035},
  {'height': 1761.54944,
   'width': 1094.66162,
   'x': 2245.91208,
   'y': 591.20528}],
 'label': {'opacity_0': [1.0, 789.28836, 582.43035, 1815.94498, 2499.73327],
  'opacity_1': [1.0, 2245.91208, 591.20528, 3340.5737, 2352.75472]}}

In [9]:
bboxes = A_metadata['boxes']

In [10]:
bbox = bboxes[0]

In [11]:
bbox

{'height': 1917.30292, 'width': 1026.65662, 'x': 789.28836, 'y': 582.43035}

In [12]:
from master_lib.image_processing import processing

In [13]:
img = processing.load_dicom_image(A)

In [14]:
cropp_img = processing.cropp_image_bbox_dict(img, bbox)

In [15]:
from PIL import Image
to_save = Image.fromarray(cropp_img)
to_save.save(os.path.join(master_img_path, os.path.basename(keys[0])) + 'cropp_from_functions.png')

In [16]:
hyperparameters_dict = {}
hyperparameters_dict['batch_size'] = 10
hyperparameters_dict['patch_size'] = (512, 512)
hyperparameters_dict['epochs'] = 100
hyperparameters_dict['conv_kernel'] = [3, 3]
hyperparameters_dict['max_pool_kernel'] = [2, 2]
hyperparameters_dict['learning_rate'] = 0.001

files_and_directories.save_json(hyperparameters_dict, r'C:\Users\MikołajStryja\Documents\Studia\master\hyperparameters.json')

In [17]:
def harmonic(x, y):
    return 2*x*y/(x+y)

In [19]:
harmonic(0.658, 0.589)

0.6215910184442663

In [30]:
imgs = 2000
negative = 1285
positive = imgs - 1285
TN = negative - 200
TP = positive - 158
FP = 200
FN = 158
conf_matrix = np.array([[TP, FP], [FN, TN]])

In [31]:
conf_matrix

array([[ 557,  200],
       [ 158, 1085]])

In [32]:
acc = (conf_matrix[0, 0] + conf_matrix[1, 1])/conf_matrix.sum()

In [33]:
acc

0.821

In [36]:
precison = TP/(TP + FP)
precison

0.7357992073976222

In [40]:
recall = TP/(TP + FN)
recall

0.779020979020979

In [39]:
F1 = harmonic(precison, recall)
F1

0.7567934782608695

In [43]:
TP = positive - 87
FP = negative - 45
FN = 45
FP = 87
efficient_net_conf_matrix = np.array([[TP, FP], [FN, TN]])

In [44]:
efficient_net_conf_matrix

array([[ 628,   87],
       [  45, 1085]])

In [45]:
efficient_net_conf_matrix[1,0]

45

In [46]:
def return_metrics(conf_matrix: np.ndarray):
    acc = (conf_matrix[0, 0] + conf_matrix[1, 1])/conf_matrix.sum()
    recall = conf_matrix[0, 0]/(conf_matrix[0, 0] + conf_matrix[1, 0])
    precison = conf_matrix[0,0]/(conf_matrix[0,0] + conf_matrix[0,1])
    F1 = harmonic(recall, precison)
    return (acc, recall, precison, F1)

In [47]:
metrics = return_metrics(efficient_net_conf_matrix)

In [48]:
metrics

(0.9284552845528455,
 0.9331352154531947,
 0.8783216783216783,
 0.9048991354466859)